In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import f_oneway
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

# Descargas necesarias para NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Cargar los conjuntos de datos
ruta_train = 'C:/Users/Marcio Pineda/Documents/Archivos Python/datasets/traincase.csv'
ruta_test = 'C:/Users/Marcio Pineda/Documents/Archivos Python/datasets/testcase.csv'
df_train = pd.read_csv(ruta_train)
df_test = pd.read_csv(ruta_test)

# Definir funciones de preprocesamiento de texto
def preprocess_text(text):
    # Tokenización
    tokens = word_tokenize(text.lower())
    # Eliminación de stopwords y puntuación
    tokens = [token for token in tokens if token not in stopwords.words('english') and token not in string.punctuation]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Preprocesamiento de la columna 'Keyword'
df_train['Preprocessed Keyword'] = df_train['Keyword'].apply(preprocess_text)

# Vectorización TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=600)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_train['Preprocessed Keyword'])

# Modelado LDA
lda_model = LatentDirichletAllocation(n_components=10, random_state=42)
lda_model.fit(tfidf_matrix)

# Asignar a cada muestra el tópico más probable de LDA
df_train['Topic'] = lda_model.transform(tfidf_matrix).argmax(axis=1)

# Convertir la columna 'Topic' en variables dummy
df_train = pd.get_dummies(df_train, columns=['Topic'], drop_first=True)

# Definir kmeans
kmeans = KMeans(n_clusters=10, random_state=42)
# Ajustar el modelo KMeans
kmeans.fit(tfidf_matrix)

def preprocess_df_general(df, kmeans):
    # Limpiar columnas numéricas, excepto 'Clicks'
    numeric_cols = ['Search Engine Bid', 'Avg. Pos.', 'Avg. Cost per Click', 'Impressions']
    for col in numeric_cols:
        if df[col].dtype == object:
            df[col] = pd.to_numeric(df[col].str.replace('$', '').str.replace(',', ''), errors='coerce')
    df['Impressions'].fillna(df['Impressions'].median(), inplace=True)

    # Procesamiento que aplica tanto al conjunto de entrenamiento como al de prueba
    keywords_tfidf = tfidf_vectorizer.transform(df['Keyword'].str.lower())
    keyword_clusters = kmeans.predict(keywords_tfidf)
    df['Keyword Cluster'] = keyword_clusters
    df['Interaction'] = df['Keyword'].astype(str) + '_' + df['Match Type'].astype(str)
    bin_edges = [0, 100, 1000, 10000, np.inf]
    bin_labels = [1, 2, 3, 4]
    df['Impressions Category'] = pd.cut(df['Impressions'], bins=bin_edges, labels=bin_labels, right=False).cat.add_categories([0]).fillna(0).astype(int)
    
    return df

def preprocess_df_train(df, kmeans):
    df = preprocess_df_general(df, kmeans)
    # Limpiar y convertir 'Clicks' a numérico solo para el conjunto de entrenamiento
    df['Clicks'] = pd.to_numeric(df['Clicks'].str.replace(',', ''), errors='coerce')
    return df

# Preprocesamiento de datos
df_train_cleaned = preprocess_df_train(df_train.copy(), kmeans)
df_test_cleaned = preprocess_df_general(df_test.copy(), kmeans)

# Transformaciones logarítmicas
df_train_cleaned['Log_Impressions'] = np.log1p(df_train_cleaned['Impressions'])
# También puedes hacer lo mismo para df_test_cleaned si es necesario

# Características polinómicas
df_train_cleaned['Search_Engine_Bid_Squared'] = df_train_cleaned['Search Engine Bid'] ** 2
df_train_cleaned['Impressions_Cubed'] = df_train_cleaned['Impressions'] ** 3
# También puedes crear más características polinómicas según sea necesario

# Actualizar las características seleccionadas
selected_features = ['Search Engine Bid', 'Impressions Category', 'Avg. Pos.', 'Keyword Cluster',
                     'Log_Impressions', 'Search_Engine_Bid_Squared', 'Impressions_Cubed']

X_train_cleaned = df_train_cleaned[selected_features]
y_train_cleaned = df_train_cleaned['Clicks'].astype(float)
X_train_cleaned.fillna(0, inplace=True)

# Ajustar el modelo de regresión lineal con las nuevas características
model_cleaned = RandomForestRegressor(n_estimators=100, random_state=42)
model_cleaned.fit(X_train_cleaned, y_train_cleaned)

# Realizar cross-validation con el modelo de regresión lineal actualizado
cv_scores_cleaned = cross_val_score(model_cleaned, X_train_cleaned, y_train_cleaned, cv=5, scoring='neg_mean_squared_error')
cv_rmse_cleaned = np.sqrt(-cv_scores_cleaned)
cv_rmse_cleaned_mean = cv_rmse_cleaned.mean()

print("RMSE promedio del modelo limpio con transformaciones logarítmicas y características polinómicas:", cv_rmse_cleaned_mean)

# Análisis de Valores Atípicos
# Investigar los casos de valores atípicos para determinar su naturaleza
outliers = df_train_cleaned[(np.abs(df_train_cleaned['Clicks'] - df_train_cleaned['Clicks'].mean()) > (3 * df_train_cleaned['Clicks'].std()))]
print("Casos de valores atípicos:")
print(outliers)

# Evaluación Estadística
# Prueba de ANOVA para determinar si las diferencias en los 'Clicks' entre los tópicos son significativas
anova_result = f_oneway(
    df_train_cleaned[df_train_cleaned['Topic_1'] == 1]['Clicks'],
    df_train_cleaned[df_train_cleaned['Topic_2'] == 1]['Clicks'],
    df_train_cleaned[df_train_cleaned['Topic_3'] == 1]['Clicks']
)

print("Resultados de ANOVA:", anova_result)


[nltk_data] Downloading package stopwords to C:\Users\Marcio
[nltk_data]     Pineda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Marcio
[nltk_data]     Pineda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Marcio
[nltk_data]     Pineda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\Marcio Pineda\AppData\Local\Temp\ipykernel_36036\314942815.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_cleaned.fillna(0, inplace=True)


RMSE promedio del modelo limpio con transformaciones logarítmicas y características polinómicas: 674.388945596839
Casos de valores atípicos:
      entry_id     Publisher Name          Keyword Match Type  \
5      mkt_007        Google - US       air france      Broad   
184   mkt_1082  Overture - Global    france travel   Advanced   
209   mkt_1105      Overture - US   airline ticket   Standard   
475   mkt_1349    Google - Global     [air france]      Exact   
592   mkt_1459        Google - US   air france com      Broad   
676   mkt_1536         Yahoo - US        airfrance   Advanced   
685   mkt_1544        Google - US       air france      Broad   
982   mkt_1818      Overture - US        airfrance   Standard   
983   mkt_1819      Overture - US    europe travel   Standard   
1314  mkt_2127        Google - US  flight to paris      Broad   
1453  mkt_2255         Yahoo - US        airfrance   Advanced   
1685  mkt_2468       MSN - Global       air france      Broad   
2084  mkt_2839

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Modelos RandomForestRegressor
model_rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42)

# Variables predictoras y variable objetivo
X = df_train_cleaned[['Search_Engine_Bid_Squared', 'Impressions_Cubed' ]]
y = df_train_cleaned['Clicks']

# Validación cruzada con RandomForestRegressor
cv_scores_rf = cross_val_score(model_rf, X, y, cv=5, scoring='neg_mean_squared_error')
cv_rmse_rf = np.sqrt(-cv_scores_rf)
cv_rmse_rf_mean = cv_rmse_rf.mean()

print("RMSE promedio del modelo RandomForestRegressor:", cv_rmse_rf_mean)

# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar el modelo RandomForestRegressor con todos los datos de entrenamiento
model_rf.fit(X_train, y_train)

# Calcular el RMSE en el conjunto de entrenamiento
train_predictions_rf = model_rf.predict(X_train)
train_rmse_rf = np.sqrt(mean_squared_error(y_train, train_predictions_rf))
print("RMSE en el conjunto de entrenamiento del modelo RandomForestRegressor:", train_rmse_rf)

# Calcular el RMSE en el conjunto de validación
valid_predictions_rf = model_rf.predict(X_valid)
valid_rmse_rf = np.sqrt(mean_squared_error(y_valid, valid_predictions_rf))
print("RMSE en el conjunto de validación del modelo RandomForestRegressor:", valid_rmse_rf)

# Verificar sobreajuste comparando el RMSE en el conjunto de entrenamiento y el conjunto de validación
if train_rmse_rf < valid_rmse_rf:
    print("El modelo RandomForestRegressor podría estar sobreajustado.")


RMSE promedio del modelo RandomForestRegressor: 742.663106047669
RMSE en el conjunto de entrenamiento del modelo RandomForestRegressor: 307.86615099736537
RMSE en el conjunto de validación del modelo RandomForestRegressor: 861.122255773637
El modelo RandomForestRegressor podría estar sobreajustado.


In [24]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Define los modelos de regresión lineal
model_lr = LinearRegression()
model_ridge = Ridge(alpha=1.0)  # Puedes ajustar el parámetro alpha según sea necesario
model_lasso = Lasso(alpha=1.0)  # Puedes ajustar el parámetro alpha según sea necesario
model_elasticnet = ElasticNet(alpha=1.0, l1_ratio=0.5)  # Puedes ajustar los parámetros alpha y l1_ratio según sea necesario

# Variables predictoras y variable objetivo
X_train = df_train_cleaned[['Search_Engine_Bid_Squared', 'Impressions_Cubed']]
y_train = df_train_cleaned['Clicks']

# Validación cruzada con modelos de regresión lineal
cv_scores_lr = cross_val_score(model_lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse_lr = np.sqrt(-cv_scores_lr)
cv_rmse_lr_mean = cv_rmse_lr.mean()

print("RMSE promedio del modelo LinearRegression:", cv_rmse_lr_mean)

# Validación cruzada con Ridge Regression
cv_scores_ridge = cross_val_score(model_ridge, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse_ridge = np.sqrt(-cv_scores_ridge)
cv_rmse_ridge_mean = cv_rmse_ridge.mean()

print("RMSE promedio del modelo Ridge Regression:", cv_rmse_ridge_mean)

# Validación cruzada con Lasso Regression
cv_scores_lasso = cross_val_score(model_lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse_lasso = np.sqrt(-cv_scores_lasso)
cv_rmse_lasso_mean = cv_rmse_lasso.mean()

print("RMSE promedio del modelo Lasso Regression:", cv_rmse_lasso_mean)

# Validación cruzada con ElasticNet Regression
cv_scores_elasticnet = cross_val_score(model_elasticnet, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse_elasticnet = np.sqrt(-cv_scores_elasticnet)
cv_rmse_elasticnet_mean = cv_rmse_elasticnet.mean()

print("RMSE promedio del modelo ElasticNet Regression:", cv_rmse_elasticnet_mean)



RMSE promedio del modelo LinearRegression: 908.9011181672895
RMSE promedio del modelo Ridge Regression: 887.6331537622891
RMSE promedio del modelo Lasso Regression: 887.6311515189813
RMSE promedio del modelo ElasticNet Regression: 887.6210503863997


C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=6.03867e-32): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=3.74076e-32): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=3.40261e-32): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=6.69695e-32): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Marcio 

In [30]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Asegúrate de aplicar las mismas transformaciones al conjunto de test
df_test_cleaned['Log_Impressions'] = np.log1p(df_test_cleaned['Impressions'])
df_test_cleaned['Search_Engine_Bid_Squared'] = df_test_cleaned['Search Engine Bid'] ** 2
df_test_cleaned['Impressions_Cubed'] = df_test_cleaned['Impressions'] ** 3

# Actualizar las características seleccionadas incluyendo las nuevas
selected_features_extended = ['Search_Engine_Bid_Squared', 'Impressions_Cubed']

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(
    df_train_cleaned[selected_features_extended],
    df_train_cleaned['Clicks'],
    test_size=0.3,
    random_state=42
)

# Definir el espacio de hiperparámetros para explorar
random_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_features': ['auto', 'sqrt']
}

# Inicializar el modelo de RandomForest
rf = RandomForestRegressor(random_state=42)

# Configurar la búsqueda aleatoria con validación cruzada
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=random_grid,
    n_iter=100,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

# Ajustar la búsqueda aleatoria
rf_random.fit(X_train, y_train)

# Mejores hiperparámetros encontrados
best_random_params = rf_random.best_params_
print("Mejores hiperparámetros de la búsqueda aleatoria:", best_random_params)

# Ajustar el modelo con los mejores hiperparámetros en el conjunto de entrenamiento completo
best_model_random = rf_random.best_estimator_

# Calcular el RMSE en el conjunto de validación utilizando el mejor modelo de la búsqueda aleatoria
valid_predictions_random = best_model_random.predict(X_valid)
valid_rmse_random = np.sqrt(mean_squared_error(y_valid, valid_predictions_random))

print("RMSE en el conjunto de validación con el mejor modelo de la búsqueda aleatoria:", valid_rmse_random)




Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
156 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_co

Mejores hiperparámetros de la búsqueda aleatoria: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50}
RMSE en el conjunto de validación con el mejor modelo de la búsqueda aleatoria: 802.2676061974535


In [31]:
# Ajustar el modelo con los mejores hiperparámetros en el conjunto de entrenamiento completo
best_model_random.fit(df_train_cleaned[selected_features_extended], df_train_cleaned['Clicks'])

# Calcular el RMSE en el conjunto de entrenamiento
train_predictions_random = best_model_random.predict(df_train_cleaned[selected_features_extended])
train_rmse_random = np.sqrt(mean_squared_error(df_train_cleaned['Clicks'], train_predictions_random))
print("RMSE en el conjunto de entrenamiento con el mejor modelo de la búsqueda aleatoria:", train_rmse_random)

# Calcular el RMSE en el conjunto de validación utilizando el mejor modelo de la búsqueda aleatoria
valid_predictions_random = best_model_random.predict(X_valid)
valid_rmse_random = np.sqrt(mean_squared_error(y_valid, valid_predictions_random))
print("RMSE en el conjunto de validación con el mejor modelo de la búsqueda aleatoria:", valid_rmse_random)

# Verificar sobreajuste comparando el RMSE en el conjunto de entrenamiento y el conjunto de validación
if train_rmse_random < valid_rmse_random:
    print("El modelo con los mejores hiperparámetros podría estar sobreajustado.")


RMSE en el conjunto de entrenamiento con el mejor modelo de la búsqueda aleatoria: 474.1711966070389
RMSE en el conjunto de validación con el mejor modelo de la búsqueda aleatoria: 416.7297870039349


In [33]:
import pandas as pd

# Asegúrate de que el modelo ElasticNet esté definido con los hiperparámetros seleccionados
model_elasticnet = ElasticNet(alpha=1.0, l1_ratio=0.5)

# Entrena el modelo ElasticNet con todo el conjunto de entrenamiento
model_elasticnet.fit(X_train, y_train)

# Aplica las mismas transformaciones al conjunto de test (si aún no se han aplicado)
df_test_cleaned['Log_Impressions'] = np.log1p(df_test_cleaned['Impressions'])
df_test_cleaned['Search_Engine_Bid_Squared'] = df_test_cleaned['Search Engine Bid'] ** 2
df_test_cleaned['Impressions_Cubed'] = df_test_cleaned['Impressions'] ** 3

# Realiza predicciones sobre el conjunto de test
y_pred_test_elasticnet = model_elasticnet.predict(df_test_cleaned[['Search_Engine_Bid_Squared', 'Impressions_Cubed']])

# Crear el DataFrame para el submission
submission_elasticnet = pd.DataFrame({
    'entry_id': df_test_cleaned['entry_id'],  # Asegúrate de que 'entry_id' está en el conjunto de test
    'Clicks': y_pred_test_elasticnet
})

# Exporta el DataFrame a un archivo CSV para el submission
submission_filename_elasticnet = 'elasticnet_submission.csv'
submission_elasticnet.to_csv(submission_filename_elasticnet, index=False)

print(f"Archivo de submission creado: {submission_filename_elasticnet}")


Archivo de submission creado: elasticnet_submission.csv


C:\Users\Marcio Pineda\Documents\Python\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.667e+09, tolerance: 3.901e+05
  model = cd_fast.enet_coordinate_descent(


In [ ]:
# Entrenar el modelo XGBoost con todo el conjunto de datos de entrenamiento
best_xgb_model.fit(X_train_pca, y_train)

# Aplicar las mismas transformaciones al conjunto de datos de prueba
X_test_pca = pca.transform(X_test_poly)

# Realizar predicciones sobre el conjunto de datos de prueba utilizando el modelo entrenado
y_pred_test_xgb = best_xgb_model.predict(X_test_pca)

# Crear el DataFrame para el envío
submission_xgb = pd.DataFrame({
    'entry_id': df_test['entry_id'],  # Asegúrate de que 'entry_id' está en el conjunto de prueba
    'Clicks': y_pred_test_xgb
})

# Exportar el DataFrame a un archivo CSV para el envío
submission_filename_xgb = 'xgb_submission.csv'
submission_xgb.to_csv(submission_filename_xgb, index=False)

print(f"Archivo de submission creado: {submission_filename_xgb}")
